Industry Classification / Data Exploration

In [45]:
import pandas as pd

#Importing test data
labeled = pd.read_csv('../data/train.csv')

In [46]:
labeled.head()

,customer_id,Industry Label (SIC),display_name,keywords
0,649df00d04a03606c70c38d4,Education,Eict Institute Limited,"[ ""Industrial artisans training "", ""Skills dev..."
1,64c25dbe215a3344ed1adc26,Information & Communication,SwiftXR 3D AR VR,"[ ""3D "", ""Augmented Reality "", ""Virtual Realit..."
2,64c2805a53a41a3b816cc3c5,Wholesale & Retail Trade,Zamar Solutions,"[ ""Battery in lagos "", ""Inverter in lagos "", ""..."
3,64c285a4775bb52fd268c188,Wholesale & Retail Trade,Afiari,"[ ""Groceries "", ""Supermarket "", ""Food delivery..."
4,64c3b14d8d37c1784d601049,Health & Social Work,Africa Medical Marketplace,"[ ""Medical Equipment Africa "", ""Healthcare Sup..."


In [47]:
labeled.shape

(414, 4)

In [48]:
labeled['keywords'].isna().sum()

0

In [49]:
# Convert keyword list (stored as string) into usable text
labeled['keywords_clean'] = labeled['keywords'].astype(str).str.strip("[]").str.replace('"', '').str.replace("'", '')
labeled['text'] = labeled['display_name'] + ' ' + labeled['keywords_clean']


In [50]:
labeled.head()

,customer_id,Industry Label (SIC),display_name,keywords,keywords_clean,text
0,649df00d04a03606c70c38d4,Education,Eict Institute Limited,"[ ""Industrial artisans training "", ""Skills dev...","Industrial artisans training , Skills develop...",Eict Institute Limited Industrial artisans t...
1,64c25dbe215a3344ed1adc26,Information & Communication,SwiftXR 3D AR VR,"[ ""3D "", ""Augmented Reality "", ""Virtual Realit...","3D , Augmented Reality , Virtual Reality , AR...","SwiftXR 3D AR VR 3D , Augmented Reality , Vi..."
2,64c2805a53a41a3b816cc3c5,Wholesale & Retail Trade,Zamar Solutions,"[ ""Battery in lagos "", ""Inverter in lagos "", ""...","Battery in lagos , Inverter in lagos , Solar ...","Zamar Solutions Battery in lagos , Inverter ..."
3,64c285a4775bb52fd268c188,Wholesale & Retail Trade,Afiari,"[ ""Groceries "", ""Supermarket "", ""Food delivery...","Groceries , Supermarket , Food delivery , Sho...","Afiari Groceries , Supermarket , Food delive..."
4,64c3b14d8d37c1784d601049,Health & Social Work,Africa Medical Marketplace,"[ ""Medical Equipment Africa "", ""Healthcare Sup...","Medical Equipment Africa , Healthcare Supplie...",Africa Medical Marketplace Medical Equipment...


In [51]:
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()
labeled['industry_encoded'] = le.fit_transform(labeled['Industry Label (SIC)'])

In [54]:
labeled

,customer_id,Industry Label (SIC),display_name,keywords,keywords_clean,text,industry_encoded
0,649df00d04a03606c70c38d4,Education,Eict Institute Limited,"[ ""Industrial artisans training "", ""Skills dev...","Industrial artisans training , Skills develop...",Eict Institute Limited Industrial artisans t...,5
1,64c25dbe215a3344ed1adc26,Information & Communication,SwiftXR 3D AR VR,"[ ""3D "", ""Augmented Reality "", ""Virtual Realit...","3D , Augmented Reality , Virtual Reality , AR...","SwiftXR 3D AR VR 3D , Augmented Reality , Vi...",8
2,64c2805a53a41a3b816cc3c5,Wholesale & Retail Trade,Zamar Solutions,"[ ""Battery in lagos "", ""Inverter in lagos "", ""...","Battery in lagos , Inverter in lagos , Solar ...","Zamar Solutions Battery in lagos , Inverter ...",14
3,64c285a4775bb52fd268c188,Wholesale & Retail Trade,Afiari,"[ ""Groceries "", ""Supermarket "", ""Food delivery...","Groceries , Supermarket , Food delivery , Sho...","Afiari Groceries , Supermarket , Food delive...",14
4,64c3b14d8d37c1784d601049,Health & Social Work,Africa Medical Marketplace,"[ ""Medical Equipment Africa "", ""Healthcare Sup...","Medical Equipment Africa , Healthcare Supplie...",Africa Medical Marketplace Medical Equipment...,7
...,...,...,...,...,...,...,...
409,63f73ac726c21a3f9663b8f5,Information & Communication,Infomoby,"[ ""Marketing "", ""Digital marketing "", ""Social ...","Marketing , Digital marketing , Social media ...","Infomoby Marketing , Digital marketing , Soc...",8
410,6420f32dc7a4f53b03202787,Arts & Entertainment,Gold N Black Stone Empire,"[ ""Tailor Made "", ""Exclusive "", ""Affordable "",...","Tailor Made , Exclusive , Affordable , Tailor...","Gold N Black Stone Empire Tailor Made , Excl...",3
411,6424397bc8d72f72342049e6,Accommodation & Food Services,NaN,"[ ""Best coffee shop near me "", ""Best coffee in...","Best coffee shop near me , Best coffee in San...",NaN,0
412,6425d989526926164e6901ca,Wholesale & Retail Trade,Naturiste Organic Oils,"[ ""Hair oil "", ""Hair growth oil "", ""Grow long ...","Hair oil , Hair growth oil , Grow long hair ,...","Naturiste Organic Oils Hair oil , Hair growt...",14
